In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
df=pd.read_pickle("/content/drive/My Drive/first_pickle.pkl")
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,3/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,3/17/2014,Closed with explanation,Yes,No,759217
1,10/1/2016,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,10/5/2016,Closed with explanation,Yes,No,2141773
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No,2163100
3,6/8/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,6/10/2014,Closed with explanation,Yes,Yes,885638
4,9/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,9/13/2014,Closed with explanation,Yes,Yes,1027760


In [0]:
df=df[["Consumer complaint narrative","Product"]]

In [0]:
df=df.dropna()

In [6]:
df["Product"].value_counts()

Debt collection                                                                 47915
Mortgage                                                                        36582
Credit reporting                                                                31592
Credit card                                                                     18842
Bank account or service                                                         14888
Credit reporting, credit repair services, or other personal consumer reports    14671
Student loan                                                                    13304
Consumer Loan                                                                    9474
Credit card or prepaid card                                                      3355
Checking or savings account                                                      2142
Payday loan                                                                      1748
Money transfers                                       

In [0]:
df["Product"]=df["Product"].replace("Credit reporting","Credit reporting, credit repair services, or other personal consumer reports")

In [0]:
df["Product"]=df["Product"].replace("Credit card","Credit card or prepaid card")

In [0]:
df["Product"]=df["Product"].replace("Prepaid card","Credit card or prepaid card")
df["Product"]=df["Product"].replace("Virtual currency","Money transfer, virtual currency, or money service")

In [0]:
df["Product"]=df["Product"].replace("Payday loan","Payday loan, title loan, or personal loan")

In [0]:
df["Product"]=df["Product"].replace("Money transfers","Money transfer, virtual currency, or money service")

In [12]:
df["Product"].value_counts()

Debt collection                                                                 47915
Credit reporting, credit repair services, or other personal consumer reports    46263
Mortgage                                                                        36582
Credit card or prepaid card                                                     23648
Bank account or service                                                         14888
Student loan                                                                    13304
Consumer Loan                                                                    9474
Payday loan, title loan, or personal loan                                        2445
Money transfer, virtual currency, or money service                               2196
Checking or savings account                                                      2142
Vehicle loan or lease                                                             821
Other financial service                               

In [0]:
le=LabelEncoder()

In [0]:
df["Product"]=le.fit_transform(df["Product"])

In [15]:
df.head()

,Consumer complaint narrative,Product
1,I have outdated information on my credit repor...,4
2,I purchased a new car on XXXX XXXX. The car de...,2
7,An account on my credit report has a mistaken ...,4
12,This company refuses to provide me verificatio...,5
16,This complaint is in regards to Square Two Fin...,5


In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
lancaster_stemmer = LancasterStemmer()
stemmer = LancasterStemmer()

In [0]:
data=df["Consumer complaint narrative"].tolist()

In [19]:
import nltk
nltk.download('punkt')
final_corpus=[]
for i in range(len(data)):
    text=word_tokenize(data[i].lower())
    text = [t for t in text if len(t)>3]
    text=[lancaster_stemmer.stem(w) for w in text if w not in stuff_to_be_removed]
    sent=" ".join(text)
    final_corpus.append(sent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
new_df=pd.DataFrame(final_corpus,columns=["sentence"])

In [0]:
labels=df["Product"].tolist()

In [0]:
new_df["Product"]=labels

In [0]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [0]:
tfidf=TfidfVectorizer(stop_words='english')

In [0]:
vector=tfidf.fit_transform(new_df["sentence"])

In [0]:
X=vector
y=new_df["Product"]

In [0]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.25,random_state=0)

In [28]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
sm=SMOTE(random_state=0)

In [0]:
X_train,y_train=sm.fit_sample(X_train,y_train)

In [0]:
nb=MultinomialNB()

In [32]:
nb.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred=nb.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score,classification_report

In [35]:
accuracy_score(y_test,y_pred)

0.7619266697337628

In [36]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64      3776
           1       0.20      0.49      0.28       529
           2       0.51      0.46      0.48      2403
           3       0.76      0.74      0.75      5781
           4       0.84      0.80      0.82     11531
           5       0.86      0.72      0.79     12071
           6       0.55      0.76      0.64       571
           7       0.90      0.91      0.91      9105
           8       0.05      0.23      0.08        73
           9       0.28      0.67      0.39       603
          10       0.80      0.89      0.84      3333
          11       0.09      0.39      0.15       217

    accuracy                           0.76     49993
   macro avg       0.54      0.64      0.56     49993
weighted avg       0.80      0.76      0.77     49993



In [0]:
rfc=RandomForestClassifier(criterion='gini',max_depth=63,random_state=0)

In [38]:
rfc.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=63, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred_rfc=rfc.predict(X_test)

In [40]:
accuracy_score(y_test,y_pred_rfc)

0.717940511671634

In [41]:
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59      3776
           1       0.19      0.13      0.16       529
           2       0.43      0.35      0.39      2403
           3       0.62      0.60      0.61      5781
           4       0.76      0.84      0.80     11531
           5       0.74      0.75      0.74     12071
           6       0.49      0.42      0.45       571
           7       0.84      0.83      0.84      9105
           8       0.31      0.05      0.09        73
           9       0.29      0.29      0.29       603
          10       0.77      0.79      0.78      3333
          11       0.18      0.04      0.07       217

    accuracy                           0.72     49993
   macro avg       0.52      0.47      0.48     49993
weighted avg       0.71      0.72      0.71     49993



In [0]:
import pandas as pd
df = pd.DataFrame()
df['text'] = final_corpus

In [121]:
df["text"].head(3)

0    outd inform credit report prevy disput remov i...
1    purchas xxxx xxxx deal cal cit bank payoff loa...
2    account credit report mistak dat mail debt val...
Name: text, dtype: object

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(df['text'].head(3))
X = vector

In [0]:
from sklearn.decomposition import TruncatedSVD
lsa=TruncatedSVD(n_components=5,n_iter=100)

In [81]:
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=100,
             random_state=None, tol=0.0)

In [117]:
for i in range(0,3):
    ranking_topic=lsa.components_[i].tolist()
    words=tfidf.get_feature_names()
    df=pd.DataFrame(list(zip(words,ranking_topic)),columns=["Words","Ranking"])
    print(df.sort_values(["Ranking"],ascending=False).head(5))

      Words   Ranking
65      let  0.415798
136    xxxx  0.381975
42      exp  0.240253
60   inform  0.209668
98   report  0.209668
      Words   Ranking
26   credit  0.313333
60   inform  0.313333
98   report  0.313333
107     sev  0.206270
97    remov  0.206270
       Words   Ranking
65       let  0.456764
42       exp  0.282460
99    repres  0.169476
70      mail  0.113055
117  suspicy  0.112984


In [0]:
new_final_corpus=[]
for i in range(len(data)):
    text=word_tokenize(data[i].lower())
    text = [t for t in text if len(t)>3]
    text=" ".join(lancaster_stemmer.stem(w) for w in text if w not in stuff_to_be_removed)
    new_final_corpus.append(text)

In [0]:
import pandas as pd
df_lda = pd.DataFrame()
df_lda['text'] = new_final_corpus

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(df_lda['text'].head(3))
X = vector

In [0]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda=LDA(n_components=3)

In [102]:
lda.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [118]:
display_topics(lda,tfidf.get_feature_names(),5)

Topic 0:
xxxx report inform credit check
Topic 1:
let exp repres receiv mail
Topic 2:
disput report credit inform receiv


In [119]:
df_lda.head()

,text
0,outd inform credit report prevy disput remov i...
1,purchas xxxx xxxx deal cal cit bank payoff loa...
2,account credit report mistak dat mail debt val...
3,company refus provid ver valid debt right fdcp...
4,complaint regard squ fin ref cfpb cas numb xxx...
